In [1]:
import os,subprocess
workdir='/mnt/Data1/ribli/expression_code/modelling/'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)

#theano gpu
os.environ['THEANO_FLAGS']='device=gpu'

#import sys
#sys.path.append('../my_modules')
#from loading_utils import read_my_data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.style.use('ggplot')
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import gc

def read_my_data(fname,**kwargs):
    """Load my data from file into np.arrays.
    
    I had to use garbage collector, because pandas read_csv leaves garbage around.
    """
    
    #load data
    print "Loading data... "
    x=pd.read_csv(fname,sep='\t',header=None)
    
    # for some reason not everything is cleaned up
    #when using the pandas read_csv
    gc.collect()
    
    #probe_id=x[0]
    #y=x.iloc[:,-1].values.astype(np.int8)
    #x=x.iloc[:,1:-1].values.astype(np.int8)
    #return probe_id,x,y
    
    return x[0],x.iloc[:,1:-1].values.astype(np.int8),x.iloc[:,-1].values.astype(np.float64)

In [3]:
train_id,train_x,train_y = read_my_data(fname='../prepare_data/naive_feat_vect.csv')
print train_x.shape

Loading data... 
(234044, 600)


In [4]:
#make it image like
train_x=train_x.reshape((-1,1,600,1))

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D,MaxPooling2D

input_dim=train_x.shape[2]
activation='relu'
loss='mean_squared_error'
optimizer='adam'
init='uniform'
pool_size=(2,1)
window_size=5
dense_n=64

model = Sequential()

#Convolution layer 1
model.add(Convolution2D(20,window_size,1, border_mode='valid',input_shape=(1,input_dim,1)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

#Convolution layer 2
model.add(Convolution2D(50,window_size,1, border_mode='valid'))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

#Dense layer
model.add(Flatten())
model.add(Dense(dense_n,activation=activation))
#model.add(Dropout(0.5))

#final layer
model.add(Dense(1, activation='linear'))

#compile model
model.compile(loss=loss,optimizer=optimizer)

Using Theano backend.
Using gpu device 0: GeForce GTX 670 (CNMeM is disabled, CuDNN not available)
/mnt/Data1/ribli/tools/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
import time
def fit_keras_model(model,train_x,train_y,test_x,test_y,validation_split=0.05):
    start=time.time()
    
    #callbacks
    best_model=ModelCheckpoint('best_model',save_best_only=True,verbose=1)
    early_stop=EarlyStopping(patience=20,verbose=1)
    
    #train it
    callb_hist=model.fit(train_x,train_y,nb_epoch = 100 ,verbose=1,
                        validation_split=validation_split,
                        callbacks=[best_model,early_stop])
    #predict
    model.load_weights('best_model')
    train_pred=model.predict(train_x,verbose=1).ravel()
    test_pred=model.predict(test_x,verbose=1).ravel()

    #check errors
    print 'train rmse:',np.mean((train_pred-train_y)**2)
    print 'test rmse:',np.mean((test_pred-test_y)**2)

    print 'It took:',time.time()-start    
    return train_pred,test_pred

In [ ]:
N_train=190000
N_test=40000

train_pred,test_pred=fit_keras_model(
    model,train_x[:N_train],train_y[:N_train],
    train_x[N_train:N_train+N_test],train_y[N_train:N_train+N_test],
    validation_split=0.2)

Train on 152000 samples, validate on 38000 samples
Epoch 1/100
 39936/152000 [======>.......................] - ETA: 59s - loss: 9.5920

### Plot results

In [ ]:
fig=plt.figure(figsize=(16,6))
fig.add_subplot(121)
plt.plot(train_pred,train_y[:N_train],'.',ms=0.5,c='b')
fig.add_subplot(122)
dump=plt.plot(test_pred,train_y[N_train:N_train+N_test],'.',ms=0.5,c='g')